In [23]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
VWAP ±2σ Intraday Backtester
- Absolute per-trade SL/TP in rupees (e.g., SL=₹2,000, TP=₹4,000)
- Trailing Stop-Loss (TSL) with activation threshold and rupee gap
- Entries on NEXT candle's OPEN after signal
- Entry window (default 09:30–14:00 IST); exits allowed till EOD
- Groww intraday charges
- Per-stock trade logs + aggregated summary
"""

import math
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple
import numpy as np
import pandas as pd
import pytz

try:
    import yfinance as yf
except Exception:
    raise SystemExit("Please: pip install yfinance pandas numpy pytz")

# =========================
# CONFIG
# =========================
@dataclass
class Config:
    # Data
    symbols: List[str] = None
    start_date: str = "2025-01-01"
    end_date:   str = "2025-11-06"
    interval:   str = "5m"

    # Strategy
    k_sigma: float = 2.0
    enable_long: bool = True
    enable_short: bool = True
    exit_on_vwap_touch: bool = True
    max_trades_per_day: Optional[int] = None

    # Entry timing
    enter_on_next_open: bool = True

    # Sizing
    capital_per_stock: float = 100000.0
    leverage: float = 5.0

    # Absolute P&L thresholds per trade (TOTAL rupees)
    sl_total_rupees: float = 2000.0
    tp_total_rupees: float = 4000.0
    both_hit_tie_policy: str = "target"  # "stop" or "target"

    # Trailing Stop-Loss (TOTAL-₹ based)
    use_trailing: bool = True
    trail_activate_total_rupees: float = 1000.0  # activate trailing once MTM ≥ this
    trail_gap_total_rupees: float = 1500.0       # trailing gap (lock this much less than peak)

    # Costs / slippage
    slippage_bps_entry: float = 2.0
    slippage_bps_exit:  float = 2.0

    # Market (India)
    tz: str = "Asia/Kolkata"
    market_open: str = "09:15"
    market_close: str = "15:30"

    # Restrict NEW ENTRIES to this window (local time)
    entry_window_start: str = "09:30"
    entry_window_end:   str = "14:00"

    # Output
    print_logs: bool = True
    save_csv: bool = True
    trades_csv: str = "trades_vwap_bands_absPnl_trail.csv"
    summary_csv: str = "summary_vwap_bands_absPnl_trail.csv"

CFG = Config(symbols=["RELIANCE.NS","TCS.NS","ICICIBANK.NS"])

# =========================
# Groww intraday charges
# =========================
def groww_intraday_charges(buy_turnover: float, sell_turnover: float) -> Dict[str, float]:
    def brokerage(turnover):
        fee = min(20.0, 0.001 * turnover)
        return max(5.0, fee)
    bro_buy  = brokerage(buy_turnover)
    bro_sell = brokerage(sell_turnover)
    exch_buy  = 0.0000297 * buy_turnover
    exch_sell = 0.0000297 * sell_turnover
    sebi_buy  = 0.000001 * buy_turnover
    sebi_sell = 0.000001 * sell_turnover
    ipft_buy  = 0.000001 * buy_turnover
    ipft_sell = 0.000001 * sell_turnover
    gst_buy  = 0.18 * (bro_buy  + exch_buy  + sebi_buy  + ipft_buy)
    gst_sell = 0.18 * (bro_sell + exch_sell + sebi_sell + ipft_sell)
    stt_sell = 0.00025 * sell_turnover
    stamp_buy = 0.00003 * buy_turnover
    total = (bro_buy + bro_sell + exch_buy + exch_sell +
             sebi_buy + sebi_sell + ipft_buy + ipft_sell +
             gst_buy + gst_sell + stt_sell + stamp_buy)
    return {"total_charges": total}

# =========================
# Helpers
# =========================
def within_market_hours(df: pd.DataFrame, tz: str, open_str: str, close_str: str) -> pd.DataFrame:
    if df.index.tz is None:
        df = df.tz_localize("UTC")
    idx_local = df.index.tz_convert(tz)
    open_t  = pd.to_datetime(open_str).time()
    close_t = pd.to_datetime(close_str).time()
    mask = (idx_local.time >= open_t) & (idx_local.time <= close_t)
    return df.loc[mask]

def add_vwap_and_bands(df: pd.DataFrame, k: float, tz: str) -> pd.DataFrame:
    df = df.tz_convert(tz).copy()
    df["session"] = df.index.date
    typical = (df["High"] + df["Low"] + df["Close"]) / 3.0
    pv = typical * df["Volume"]
    df["cum_pv"] = pv.groupby(df["session"]).cumsum()
    df["cum_v"]  = df["Volume"].groupby(df["session"]).cumsum()
    df["VWAP"]   = df["cum_pv"] / df["cum_v"]
    df["Close_std"] = df.groupby("session")["Close"].expanding().std().reset_index(level=0, drop=True)
    df["Upper"] = df["VWAP"] + k * df["Close_std"]
    df["Lower"] = df["VWAP"] - k * df["Close_std"]
    return df

def size_qty(entry_price: float, capital: float, leverage: float) -> int:
    notional = capital * leverage
    return max(0, math.floor(notional / max(1e-9, entry_price)))

def bps_slip(price: float, bps: float, side: str) -> float:
    slip = price * (bps / 10000.0)
    return price + slip if side == "buy" else price - slip

def in_entry_window(ts: pd.Timestamp, cfg: Config) -> bool:
    tlocal = ts.tz_convert(cfg.tz).time()
    start = pd.to_datetime(cfg.entry_window_start).time()
    end   = pd.to_datetime(cfg.entry_window_end).time()
    return (tlocal >= start) and (tlocal <= end)

# =========================
# Core backtest
# =========================
def fetch(symbol: str, cfg: Config) -> pd.DataFrame:
    df = yf.download(symbol, start=cfg.start_date, end=cfg.end_date,
                     interval=cfg.interval, auto_adjust=False, progress=False, multi_level_index=False)
    if df.empty: return df
    return within_market_hours(df, cfg.tz, cfg.market_open, cfg.market_close)

def build_trades_for_symbol(symbol: str, cfg: Config) -> pd.DataFrame:
    df = fetch(symbol, cfg)
    if df.empty:
        return pd.DataFrame(columns=[
            "symbol","session","side","entry_time","entry_price","qty",
            "exit_time","exit_price","gross_pnl","charges","net_pnl","exit_reason"
        ])

    df = add_vwap_and_bands(df, cfg.k_sigma, cfg.tz).dropna(subset=["VWAP","Upper","Lower"]).copy()

    # Signal on bar t, entry on bar t+1 open
    df["long_signal"]  = (df["Low"]  <= df["Lower"])  if cfg.enable_long  else False
    df["short_signal"] = (df["High"] >= df["Upper"])  if cfg.enable_short else False
    df["enter_long_next"]  = df["long_signal"].shift(1).fillna(False)
    df["enter_short_next"] = df["short_signal"].shift(1).fillna(False)

    trades = []
    in_pos = False
    side = None
    qty = 0
    entry_price = np.nan
    entry_time = None

    # Fixed absolute SL/TP price levels (from total-₹ thresholds)
    sl_price = np.nan
    tp_price = np.nan

    # Trailing context
    trailing_active = False
    trail_gap_per_share = np.nan
    best_price_since_entry = np.nan   # highest high (long) or lowest low (short)
    trailing_stop_price = np.nan      # dynamic SL

    current_session = None
    day_trade_count = 0

    for ts, r in df.iterrows():
        sess = r["session"]
        if current_session is None or sess != current_session:
            current_session = sess
            in_pos = False; side=None; qty=0
            day_trade_count = 0
            trailing_active = False
            best_price_since_entry = np.nan
            trailing_stop_price = np.nan

        is_last_bar = (df.loc[df["session"]==sess].index[-1] == ts)
        o, h, l, c, vwap = r["Open"], r["High"], r["Low"], r["Close"], r["VWAP"]

        # ---- Entry on next bar open (inside window) ----
        if not in_pos and (cfg.max_trades_per_day is None or day_trade_count < cfg.max_trades_per_day):
            if in_entry_window(ts, cfg):
                if r["enter_long_next"]:
                    ep = bps_slip(o, cfg.slippage_bps_entry, "buy")
                    q  = size_qty(ep, cfg.capital_per_stock, cfg.leverage)
                    if q > 0:
                        in_pos, side, qty = True, "LONG", q
                        entry_price, entry_time = ep, ts
                        sl_price = ep - (cfg.sl_total_rupees / q)
                        tp_price = ep + (cfg.tp_total_rupees / q)
                        # trailing init
                        trailing_active = False
                        trail_gap_per_share = cfg.trail_gap_total_rupees / q
                        best_price_since_entry = ep
                        trailing_stop_price = np.nan
                        day_trade_count += 1

                elif r["enter_short_next"]:
                    ep = bps_slip(o, cfg.slippage_bps_entry, "sell")
                    q  = size_qty(ep, cfg.capital_per_stock, cfg.leverage)
                    if q > 0:
                        in_pos, side, qty = True, "SHORT", q
                        entry_price, entry_time = ep, ts
                        sl_price = ep + (cfg.sl_total_rupees / q)
                        tp_price = ep - (cfg.tp_total_rupees / q)
                        trailing_active = False
                        trail_gap_per_share = cfg.trail_gap_total_rupees / q
                        best_price_since_entry = ep
                        trailing_stop_price = np.nan
                        day_trade_count += 1

        # ---- Exit checks ----
        if in_pos:
            # Update best in-favor price and activate/adjust trailing if needed
            if cfg.use_trailing:
                if side == "LONG":
                    # update best (highest high)
                    if h > best_price_since_entry:
                        best_price_since_entry = h
                    # compute current MTM (unrealized) in TOTAL ₹ at close
                    unrealized_total = (c - entry_price) * qty
                    if not trailing_active and unrealized_total >= cfg.trail_activate_total_rupees:
                        trailing_active = True
                        trailing_stop_price = best_price_since_entry - trail_gap_per_share
                    elif trailing_active:
                        # move stop up if peak advanced; never loosen
                        new_stop = best_price_since_entry - trail_gap_per_share
                        if np.isnan(trailing_stop_price) or (new_stop > trailing_stop_price):
                            trailing_stop_price = new_stop
                else:
                    # SHORT: update best (lowest low)
                    if l < best_price_since_entry or np.isnan(best_price_since_entry):
                        best_price_since_entry = l if not np.isnan(best_price_since_entry) else l
                        best_price_since_entry = min(best_price_since_entry, l)
                    else:
                        best_price_since_entry = min(best_price_since_entry, l)
                    unrealized_total = (entry_price - c) * qty
                    if not trailing_active and unrealized_total >= cfg.trail_activate_total_rupees:
                        trailing_active = True
                        trailing_stop_price = best_price_since_entry + trail_gap_per_share
                    elif trailing_active:
                        new_stop = best_price_since_entry + trail_gap_per_share
                        if np.isnan(trailing_stop_price) or (new_stop < trailing_stop_price):
                            trailing_stop_price = new_stop

            exit_px = None
            exit_reason = None

            # 1) Absolute SL/TP (intrabar)
            if side == "LONG":
                abs_stop_hit = l <= sl_price
                abs_targ_hit = h >= tp_price
            else:
                abs_stop_hit = h >= sl_price
                abs_targ_hit = l <= tp_price

            if abs_stop_hit and abs_targ_hit:
                if CFG.both_hit_tie_policy.lower() == "stop":
                    exit_px, exit_reason = sl_price, "SL_total₹"
                else:
                    exit_px, exit_reason = tp_price, "TP_total₹"
            elif abs_stop_hit:
                exit_px, exit_reason = sl_price, "SL_total₹"
            elif abs_targ_hit:
                exit_px, exit_reason = tp_price, "TP_total₹"

            # 2) Trailing SL (if still open)
            if exit_px is None and cfg.use_trailing and trailing_active and np.isfinite(trailing_stop_price):
                if side == "LONG" and l <= trailing_stop_price:
                    exit_px, exit_reason = trailing_stop_price, "TSL"
                elif side == "SHORT" and h >= trailing_stop_price:
                    exit_px, exit_reason = trailing_stop_price, "TSL"

            # 3) VWAP-touch (still open)
            if exit_px is None and cfg.exit_on_vwap_touch:
                if side == "LONG" and c >= vwap:
                    exit_px, exit_reason = vwap, "VWAP_touch"
                elif side == "SHORT" and c <= vwap:
                    exit_px, exit_reason = vwap, "VWAP_touch"

            # 4) EOD
            if exit_px is None and is_last_bar:
                exit_px, exit_reason = c, "EOD"

            # ---- Book trade ----
            if exit_px is not None:
                if side == "LONG":
                    exit_px = bps_slip(exit_px, cfg.slippage_bps_exit, "sell")
                    buy_turnover  = entry_price * qty
                    sell_turnover = exit_px * qty
                    gross = (exit_px - entry_price) * qty
                else:
                    exit_px = bps_slip(exit_px, cfg.slippage_bps_exit, "buy")
                    buy_turnover  = exit_px * qty
                    sell_turnover = entry_price * qty
                    gross = (entry_price - exit_px) * qty

                charges = groww_intraday_charges(buy_turnover, sell_turnover)["total_charges"]
                net = gross - charges

                trades.append({
                    "symbol": symbol,
                    "session": str(sess),
                    "side": side,
                    "entry_time": entry_time.tz_convert(CFG.tz).strftime("%Y-%m-%d %H:%M"),
                    "entry_price": round(entry_price, 2),
                    "qty": int(qty),
                    "exit_time": ts.tz_convert(CFG.tz).strftime("%Y-%m-%d %H:%M"),
                    "exit_price": round(exit_px, 2),
                    "gross_pnl": round(gross, 2),
                    "charges": round(charges, 2),
                    "net_pnl": round(net, 2),
                    "exit_reason": exit_reason,
                })

                # flat
                in_pos = False
                side = None
                qty = 0
                entry_price = np.nan
                sl_price = tp_price = np.nan
                trailing_active = False
                best_price_since_entry = np.nan
                trailing_stop_price = np.nan

    return pd.DataFrame(trades)

def run_backtest(cfg: Config) -> Tuple[pd.DataFrame, pd.DataFrame]:
    all_trades = []
    for s in cfg.symbols or []:
        tdf = build_trades_for_symbol(s, cfg)
        all_trades.append(tdf)
        if cfg.print_logs:
            print(f"\n=== {s} — {len(tdf)} trades ===")
            print("No trades." if tdf.empty else tdf.to_string(index=False))

    trades = pd.concat(all_trades, ignore_index=True) if all_trades else pd.DataFrame()
    if trades.empty:
        summary = pd.DataFrame(columns=["symbol","trades","gross_pnl","charges","net_pnl"])
        return trades, summary

    summary = (
        trades.groupby("symbol", as_index=False)
              .agg(trades=("net_pnl","count"),
                   gross_pnl=("gross_pnl","sum"),
                   charges=("charges","sum"),
                   net_pnl=("net_pnl","sum"))
              .sort_values("net_pnl", ascending=False)
    )
    totals = pd.DataFrame({
        "symbol": ["__TOTAL__"],
        "trades": [int(summary["trades"].sum())],
        "gross_pnl": [round(trades["gross_pnl"].sum(),2)],
        "charges": [round(trades["charges"].sum(),2)],
        "net_pnl": [round(trades["net_pnl"].sum(),2)],
    })
    summary = pd.concat([summary, totals], ignore_index=True)

    if cfg.print_logs:
        print("\n================ Aggregated Summary ================")
        print(summary.to_string(index=False))

    if cfg.save_csv:
        trades.to_csv(cfg.trades_csv, index=False)
        summary.to_csv(cfg.summary_csv, index=False)
        if cfg.print_logs:
            print(f"\nSaved trades:  {cfg.trades_csv}")
            print(f"Saved summary: {cfg.summary_csv}")

    return trades, summary

if __name__ == "__main__":
    run_backtest(CFG)


/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_6057/1442291187.py:163: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["enter_long_next"]  = df["long_signal"].shift(1).fillna(False)
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_6057/1442291187.py:164: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["enter_short_next"] = df["short_signal"].shift(1).fillna(False)



=== ADANIENT.NS — 28 trades ===
     symbol    session side       entry_time  entry_price  qty        exit_time  exit_price  gross_pnl  charges  net_pnl exit_reason
ADANIENT.NS 2025-10-01 LONG 2025-10-01 10:25       2497.9  200 2025-10-01 10:30     2487.90   -2000.00   223.88 -2223.88   SL_total₹
ADANIENT.NS 2025-10-01 LONG 2025-10-01 10:35       2486.1  201 2025-10-01 10:45     2476.15   -2000.00   223.93 -2223.93   SL_total₹
ADANIENT.NS 2025-10-03 LONG 2025-10-03 12:00       2577.9  193 2025-10-03 14:35     2598.63    4000.00   224.88  3775.12   TP_total₹
ADANIENT.NS 2025-10-07 LONG 2025-10-07 11:05       2591.1  192 2025-10-07 11:35     2580.68   -2000.00   223.14 -2223.14   SL_total₹
ADANIENT.NS 2025-10-07 LONG 2025-10-07 11:40       2582.0  193 2025-10-07 11:55     2571.64   -2000.00   223.44 -2223.44   SL_total₹
ADANIENT.NS 2025-10-08 LONG 2025-10-08 10:15       2535.0  197 2025-10-08 10:20     2524.85   -2000.00   223.82 -2223.82   SL_total₹
ADANIENT.NS 2025-10-08 LONG 2025-10-

/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_6057/1442291187.py:163: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["enter_long_next"]  = df["long_signal"].shift(1).fillna(False)
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_6057/1442291187.py:164: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["enter_short_next"] = df["short_signal"].shift(1).fillna(False)



=== ADANIPORTS.NS — 24 trades ===
       symbol    session side       entry_time  entry_price  qty        exit_time  exit_price  gross_pnl  charges  net_pnl exit_reason
ADANIPORTS.NS 2025-10-01 LONG 2025-10-01 10:10       1407.4  355 2025-10-01 10:30     1401.77   -2000.00   223.90 -2223.90   SL_total₹
ADANIPORTS.NS 2025-10-01 LONG 2025-10-01 10:35       1400.9  356 2025-10-01 10:45     1395.28   -2000.00   223.58 -2223.58   SL_total₹
ADANIPORTS.NS 2025-10-03 LONG 2025-10-03 11:30       1413.7  353 2025-10-03 15:25     1419.90    2188.63   224.89  1963.73         EOD
ADANIPORTS.NS 2025-10-07 LONG 2025-10-07 11:25       1402.4  356 2025-10-07 15:25     1397.60   -1708.82   223.85 -1932.67         EOD
ADANIPORTS.NS 2025-10-08 LONG 2025-10-08 11:00       1393.1  358 2025-10-08 11:05     1387.51   -2000.00   223.58 -2223.58   SL_total₹
ADANIPORTS.NS 2025-10-08 LONG 2025-10-08 11:10       1387.1  360 2025-10-08 13:05     1398.21    4000.00   225.53  3774.47   TP_total₹
ADANIPORTS.NS 2025-1

/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_6057/1442291187.py:163: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["enter_long_next"]  = df["long_signal"].shift(1).fillna(False)
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_6057/1442291187.py:164: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["enter_short_next"] = df["short_signal"].shift(1).fillna(False)


...

/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_6057/1442291187.py:163: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["enter_long_next"]  = df["long_signal"].shift(1).fillna(False)
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_6057/1442291187.py:164: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["enter_short_next"] = df["short_signal"].shift(1).fillna(False)



=== ULTRACEMCO.NS — 18 trades ===
       symbol    session side       entry_time  entry_price  qty        exit_time  exit_price  gross_pnl  charges  net_pnl exit_reason
ULTRACEMCO.NS 2025-10-01 LONG 2025-10-01 10:05      12152.0   41 2025-10-01 10:35    12103.22    -2000.0   223.40 -2223.40   SL_total₹
ULTRACEMCO.NS 2025-10-07 LONG 2025-10-07 11:05      12118.0   41 2025-10-07 15:25    12184.00     2706.0   224.26  2481.74         EOD
ULTRACEMCO.NS 2025-10-08 LONG 2025-10-08 10:00      12115.0   41 2025-10-08 11:00    12066.22    -2000.0   222.87 -2222.87   SL_total₹
ULTRACEMCO.NS 2025-10-08 LONG 2025-10-08 11:05      12051.0   41 2025-10-08 11:45    12002.22    -2000.0   221.93 -2221.93   SL_total₹
ULTRACEMCO.NS 2025-10-13 LONG 2025-10-13 11:00      12193.0   41 2025-10-13 15:25    12172.00     -861.0   224.33 -1085.33         EOD
ULTRACEMCO.NS 2025-10-14 LONG 2025-10-14 11:15      12089.0   41 2025-10-14 12:05    12040.22    -2000.0   222.49 -2222.49   SL_total₹
ULTRACEMCO.NS 2025-1

/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_6057/1442291187.py:163: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["enter_long_next"]  = df["long_signal"].shift(1).fillna(False)
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_6057/1442291187.py:164: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["enter_short_next"] = df["short_signal"].shift(1).fillna(False)



=== WIPRO.NS — 21 trades ===
  symbol    session side       entry_time  entry_price  qty        exit_time  exit_price  gross_pnl  charges  net_pnl exit_reason
WIPRO.NS 2025-10-01 LONG 2025-10-01 10:50       238.00 2100 2025-10-01 12:05      239.90    4000.00   225.68  3774.32   TP_total₹
WIPRO.NS 2025-10-07 LONG 2025-10-07 10:20       243.55 2052 2025-10-07 10:55      242.58   -2000.00   223.95 -2223.95   SL_total₹
WIPRO.NS 2025-10-07 LONG 2025-10-07 11:00       242.70 2060 2025-10-07 15:25      243.53    1709.80   225.08  1484.72         EOD
WIPRO.NS 2025-10-08 LONG 2025-10-08 10:00       245.12 2039 2025-10-08 10:10      244.14   -2000.00   223.96 -2223.96   SL_total₹
WIPRO.NS 2025-10-08 LONG 2025-10-08 10:15       244.28 2046 2025-10-08 11:45      243.30   -2000.00   223.96 -2223.96   SL_total₹
WIPRO.NS 2025-10-09 LONG 2025-10-09 10:15       244.39 2045 2025-10-09 12:35      246.35    4000.00   225.68  3774.32   TP_total₹
WIPRO.NS 2025-10-13 LONG 2025-10-13 11:05       244.72 2043 

/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_6057/1442291187.py:294: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  trades = pd.concat(all_trades, ignore_index=True) if all_trades else pd.DataFrame()
